In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc
from sklearn.model_selection import KFold

import statsmodels.formula.api as smf
import statsmodels.api as sm


from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it

#Libraries for visualizing trees
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus
import time as time

In [5]:
data = pd.read_csv('listings.csv.gz')

In [6]:
data.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2384,https://www.airbnb.com/rooms/2384,20211215062239,2021-12-15,Hyde Park - Walk to University of Chicago,If you have proof of vaccination and can clim...,The apartment is less than one block from beau...,https://a0.muscache.com/pictures/9780bb37-fb54...,2613,https://www.airbnb.com/users/show/2613,...,4.98,4.95,4.94,R17000015609,f,1,0,1,0,2.27
1,7126,https://www.airbnb.com/rooms/7126,20211215062239,2021-12-15,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,17928,https://www.airbnb.com/users/show/17928,...,4.86,4.87,4.73,R18000034991,f,1,1,0,0,2.80
2,10945,https://www.airbnb.com/rooms/10945,20211215062239,2021-12-15,The Biddle House (#1),Beautiful first floor apartment in Historic Ol...,NaN,https://a0.muscache.com/pictures/58d1a420-a24b...,33004,https://www.airbnb.com/users/show/33004,...,4.72,4.97,4.56,2120297,t,8,8,0,0,0.42
3,12140,https://www.airbnb.com/rooms/12140,20211215062239,2021-12-15,Lincoln Park Guest House,Bed and Breakfast license issued by the City o...,The Guest House is in the heart of Chicago's m...,https://a0.muscache.com/pictures/55126/6a4f01b...,46734,https://www.airbnb.com/users/show/46734,...,5.00,5.00,5.00,R20000055258,f,1,0,1,0,0.09
4,22362,https://www.airbnb.com/rooms/22362,20211215062239,2021-12-15,Luxury in Chicago! 2BR/ 2Ba / Parking / BBQ,Fantastic condo in a Great Location with cover...,"Our home overlooks the park and is on a quiet,...",https://a0.muscache.com/pictures/117030/58b7f9...,85811,https://www.airbnb.com/users/show/85811,...,5.00,4.89,4.89,R17000018689,f,2,2,0,0,0.09


In [7]:
data[['Bathrooms', 'Drop', 'Delete']] = data['bathrooms_text'].str.split(' ', expand=True)

In [8]:
print(data.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [9]:
#dropping irrelevant predictors
data2 = data.drop(['Drop', 'Delete', 'listing_url', 'host_about','host_total_listings_count', 'calculated_host_listings_count','scrape_id','calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','host_since','last_scraped', 'host_url', 'picture_url', 'host_thumbnail_url', 'latitude', 'longitude', 'license', 'host_has_profile_pic'], axis = 1)

In [10]:
data2.head()

,id,name,description,neighborhood_overview,host_id,host_name,host_location,host_response_time,host_response_rate,host_acceptance_rate,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,Bathrooms
0,2384,Hyde Park - Walk to University of Chicago,If you have proof of vaccination and can clim...,The apartment is less than one block from beau...,2613,Rebecca,"Chicago, Illinois, United States",within an hour,100%,84%,...,4.99,4.98,4.99,4.98,4.98,4.95,4.94,f,2.27,1
1,7126,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",17928,Sarah,"Chicago, Illinois, United States",within an hour,100%,97%,...,4.67,4.83,4.54,4.90,4.86,4.87,4.73,f,2.80,1
2,10945,The Biddle House (#1),Beautiful first floor apartment in Historic Ol...,NaN,33004,At Home Inn,"Chicago, Illinois, United States",within an hour,94%,94%,...,4.59,4.67,4.74,4.77,4.72,4.97,4.56,t,0.42,1
3,12140,Lincoln Park Guest House,Bed and Breakfast license issued by the City o...,The Guest House is in the heart of Chicago's m...,46734,Sharon And Robert,"Chicago, Illinois, United States",NaN,NaN,100%,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,f,0.09,1
4,22362,Luxury in Chicago! 2BR/ 2Ba / Parking / BBQ,Fantastic condo in a Great Location with cover...,"Our home overlooks the park and is on a quiet,...",85811,Craig,"St. Petersburg, Florida, United States",NaN,NaN,0%,...,4.67,4.78,4.78,5.00,5.00,4.89,4.89,f,0.09,2


In [11]:
data['bathrooms_text'].str.split(' ', expand=True)

,0,1,2
0,1,shared,bath
1,1,bath,None
2,1,bath,None
3,1,private,bath
4,2,baths,None
...,...,...,...
6539,1,bath,None
6540,1,bath,None
6541,2,baths,None
6542,2,baths,None


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

In [13]:
mlb = MultiLabelBinarizer()

In [14]:
df1 = pd.DataFrame(mlb.fit_transform(data2.pop('amenities')),columns=mlb.classes_)

In [15]:
df = data2.join(df1)

In [16]:
df.head()

,id,name,description,neighborhood_overview,host_id,host_name,host_location,host_response_time,host_response_rate,host_acceptance_rate,...,q,r,s,t,u,v,w,x,y,z
0,2384,Hyde Park - Walk to University of Chicago,If you have proof of vaccination and can clim...,The apartment is less than one block from beau...,2613,Rebecca,"Chicago, Illinois, United States",within an hour,100%,84%,...,0,1,1,1,1,1,1,1,1,1
1,7126,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",17928,Sarah,"Chicago, Illinois, United States",within an hour,100%,97%,...,0,1,1,1,1,1,1,1,1,1
2,10945,The Biddle House (#1),Beautiful first floor apartment in Historic Ol...,NaN,33004,At Home Inn,"Chicago, Illinois, United States",within an hour,94%,94%,...,0,1,1,1,1,1,1,1,1,0
3,12140,Lincoln Park Guest House,Bed and Breakfast license issued by the City o...,The Guest House is in the heart of Chicago's m...,46734,Sharon And Robert,"Chicago, Illinois, United States",NaN,NaN,100%,...,0,1,1,1,1,0,1,1,1,0
4,22362,Luxury in Chicago! 2BR/ 2Ba / Parking / BBQ,Fantastic condo in a Great Location with cover...,"Our home overlooks the park and is on a quiet,...",85811,Craig,"St. Petersburg, Florida, United States",NaN,NaN,0%,...,0,1,1,1,1,1,1,1,1,0


In [17]:
datacopy = data.copy()

## Splitting amenity lists and creating dummy variables for the most common ones

In [18]:
data['amenities']

0       ["Shampoo", "Cooking basics", "Hot water", "Ir...
1       ["Stove", "Cooking basics", "Hot water", "Iron...
2       ["Shampoo", "Cable TV", "Stove", "Cooking basi...
3       ["Shampoo", "Cable TV", "Hot water", "Iron", "...
4       ["Shampoo", "Cable TV", "Stove", "Cooking basi...
                              ...                        
6539    ["Kitchen", "Washer", "Long term stays allowed...
6540                ["Long term stays allowed", "Keypad"]
6541    ["Shampoo", "Body soap", "Kitchen", "Washer", ...
6542    ["Shampoo", "Body soap", "Kitchen", "Washer", ...
6543    ["Shampoo", "Stove", "Fire pit", "Cooking basi...
Name: amenities, Length: 6544, dtype: object

In [19]:
def filtervar_shampoo(lst):
    if "Shampoo" in lst:
        return 1
    else:
        return 0

datacopy["shampoo"] = datacopy['amenities'].apply(filtervar_shampoo)

In [20]:
def filtervar_cooking(lst):
    if "Cooking basics" in lst:
        return 1
    else:
        return 0

datacopy["cooking basics"] = datacopy['amenities'].apply(filtervar_cooking)

def filtervar(lst):
    if "Hot water" in lst:
        return 1
    else:
        return 0

datacopy["hot water"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Iron" in lst:
        return 1
    else:
        return 0

datacopy["iron"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Room-darkening shades" in lst:
        return 1
    else:
        return 0

datacopy["room darkening shades"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Freezer" in lst:
        return 1
    else:
        return 0

datacopy["freezer"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Wine glasses" in lst:
        return 1
    else:
        return 0

datacopy["wine glasses"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Essentials" in lst:
        return 1
    else:
        return 0

datacopy["essentials"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Coffee maker" in lst:
        return 1
    else:
        return 0

datacopy["coffee maker"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Heating" in lst:
        return 1
    else:
        return 0

datacopy["heating"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Dishes and silverware" in lst:
        return 1
    else:
        return 0

datacopy["dishes and silverware"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Wifi" in lst:
        return 1
    else:
        return 0

datacopy["wifi"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "BBQ grill" in lst:
        return 1
    else:
        return 0

datacopy["bbq grill"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Cable TV" in lst:
        return 1
    else:
        return 0

datacopy["cable tv"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Iron" in lst:
        return 1
    else:
        return 0

datacopy["Iron"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Cooking basics" in lst:
        return 1
    else:
        return 0

datacopy["Cooking basics"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Hangers" in lst:
        return 1
    else:
        return 0

datacopy["Hangers"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Hot water" in lst:
        return 1
    else:
        return 0

datacopy["Hot water"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Private entrance" in lst:
        return 1
    else:
        return 0

datacopy["Private entrance"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Free parking on premises" in lst:
        return 1
    else:
        return 0

datacopy["Free parking on premises"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Free street parking" in lst:
        return 1
    else:
        return 0

datacopy["Free street parking"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Air conditioning" in lst:
        return 1
    else:
        return 0

datacopy["Air conditioning"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Microwave" in lst:
        return 1
    else:
        return 0

datacopy["Microwave"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Fire extinguisher" in lst:
        return 1
    else:
        return 0

datacopy["Fire extinguisher"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Shared patio or balcony" in lst:
        return 1
    else:
        return 0

datacopy["Shared patio or balcony"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Hair dryer" in lst:
        return 1
    else:
        return 0

datacopy["Hair dryer"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Stove" in lst:
        return 1
    else:
        return 0

datacopy["Stove"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Breakfast" in lst:
        return 1
    else:
        return 0

datacopy["Breakfast"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Private patio or balcony" in lst:
        return 1
    else:
        return 0

datacopy["Private patio or balcony"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Full kitchen" in lst:
        return 1
    else:
        return 0

datacopy["Full kitchen"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Private entrance" in lst:
        return 1
    else:
        return 0

datacopy["Private entrance"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Lockbox" in lst:
        return 1
    else:
        return 0

datacopy["Lockbox"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Security cameras on property" in lst:
        return 1
    else:
        return 0

datacopy["Security cameras on property"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Hot tub" in lst:
        return 1
    else:
        return 0

datacopy["Hot tub"] = datacopy['amenities'].apply(filtervar)

def filtervar(lst):
    if "Heating" in lst:
        return 1
    else:
        return 0

datacopy["Heating"] = datacopy['amenities'].apply(filtervar)

In [21]:
datacopy

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,Shared patio or balcony,Hair dryer,Stove,Breakfast,Private patio or balcony,Full kitchen,Lockbox,Security cameras on property,Hot tub,Heating
0,2384,https://www.airbnb.com/rooms/2384,20211215062239,2021-12-15,Hyde Park - Walk to University of Chicago,If you have proof of vaccination and can clim...,The apartment is less than one block from beau...,https://a0.muscache.com/pictures/9780bb37-fb54...,2613,https://www.airbnb.com/users/show/2613,...,0,1,0,1,0,0,1,0,0,1
1,7126,https://www.airbnb.com/rooms/7126,20211215062239,2021-12-15,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,17928,https://www.airbnb.com/users/show/17928,...,0,1,1,0,0,0,0,0,0,0
2,10945,https://www.airbnb.com/rooms/10945,20211215062239,2021-12-15,The Biddle House (#1),Beautiful first floor apartment in Historic Ol...,NaN,https://a0.muscache.com/pictures/58d1a420-a24b...,33004,https://www.airbnb.com/users/show/33004,...,0,1,1,0,0,0,0,0,0,1
3,12140,https://www.airbnb.com/rooms/12140,20211215062239,2021-12-15,Lincoln Park Guest House,Bed and Breakfast license issued by the City o...,The Guest House is in the heart of Chicago's m...,https://a0.muscache.com/pictures/55126/6a4f01b...,46734,https://www.airbnb.com/users/show/46734,...,0,1,0,0,0,0,0,0,0,1
4,22362,https://www.airbnb.com/rooms/22362,20211215062239,2021-12-15,Luxury in Chicago! 2BR/ 2Ba / Parking / BBQ,Fantastic condo in a Great Location with cover...,"Our home overlooks the park and is on a quiet,...",https://a0.muscache.com/pictures/117030/58b7f9...,85811,https://www.airbnb.com/users/show/85811,...,0,1,1,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6539,53812963,https://www.airbnb.com/rooms/53812963,20211215062239,2021-12-15,Quaint quiet 1 bedroom unit in Bucktown Chicago,cozy 1 bedroom in Bucktown Chicago. Close to e...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,257806893,https://www.airbnb.com/users/show/257806893,...,0,0,0,0,0,0,0,0,0,0
6540,53816167,https://www.airbnb.com/rooms/53816167,20211215062239,2021-12-15,A space that welcomes all with good energy..,The space is in a community that has a outside...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,400344330,https://www.airbnb.com/users/show/400344330,...,0,0,0,0,0,0,0,0,0,0
6541,53820517,https://www.airbnb.com/rooms/53820517,20211215062239,2021-12-15,lake and city views entire 2B,<b>License number</b><br />R12345678901,NaN,https://a0.muscache.com/pictures/4c30bcc2-1cb7...,170730783,https://www.airbnb.com/users/show/170730783,...,0,0,0,0,0,0,0,0,0,0
6542,53820855,https://www.airbnb.com/rooms/53820855,20211215062239,2021-12-15,lake and city views entire 2B,NaN,NaN,https://a0.muscache.com/pictures/4c30bcc2-1cb7...,170730783,https://www.airbnb.com/users/show/170730783,...,0,0,0,0,0,0,0,0,0,0


In [22]:
datacopy[['Bathrooms', 'Drop', 'Delete']] = datacopy['bathrooms_text'].str.split(' ', expand=True)

In [23]:
print(datacopy.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       ...
       'Shared patio or balcony', 'Hair dryer', 'Stove', 'Breakfast',
       'Private patio or balcony', 'Full kitchen', 'Lockbox',
       'Security cameras on property', 'Hot tub', 'Heating'],
      dtype='object', length=111)


In [24]:
data3 = datacopy.drop(['Drop', 'Delete', 'calendar_last_scraped', 'calendar_updated', 'neighbourhood_group_cleansed', 'host_listings_count', 'host_picture_url', 'first_review', 'last_review','host_location', 'host_response_time', 'listing_url', 'host_about','host_total_listings_count', 'availability_60', 'availability_30', 'availability_90','calculated_host_listings_count','scrape_id','calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','host_since','last_scraped', 'host_url', 'picture_url', 'host_thumbnail_url', 'latitude', 'longitude', 'license', 'host_has_profile_pic'], axis = 1)

In [25]:
print(data3.columns)

Index(['id', 'name', 'description', 'neighborhood_overview', 'host_id',
       'host_name', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_verifications',
       'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_p

In [26]:
data3.head()

,id,name,description,neighborhood_overview,host_id,host_name,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,...,Shared patio or balcony,Hair dryer,Stove,Breakfast,Private patio or balcony,Full kitchen,Lockbox,Security cameras on property,Hot tub,Heating
0,2384,Hyde Park - Walk to University of Chicago,If you have proof of vaccination and can clim...,The apartment is less than one block from beau...,2613,Rebecca,100%,84%,t,Hyde Park,...,0,1,0,1,0,0,1,0,0,1
1,7126,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",17928,Sarah,100%,97%,t,Ukrainian Village,...,0,1,1,0,0,0,0,0,0,0
2,10945,The Biddle House (#1),Beautiful first floor apartment in Historic Ol...,NaN,33004,At Home Inn,94%,94%,t,Old Town,...,0,1,1,0,0,0,0,0,0,1
3,12140,Lincoln Park Guest House,Bed and Breakfast license issued by the City o...,The Guest House is in the heart of Chicago's m...,46734,Sharon And Robert,NaN,100%,f,Lincoln Park,...,0,1,0,0,0,0,0,0,0,1
4,22362,Luxury in Chicago! 2BR/ 2Ba / Parking / BBQ,Fantastic condo in a Great Location with cover...,"Our home overlooks the park and is on a quiet,...",85811,Craig,NaN,0%,f,West Town/Noble Square,...,0,1,1,0,0,0,1,1,0,1


In [29]:
#data3.to_csv('data3.csv', index = False)